This notebook estimates throughput of pay per token and provisioned throughput models for summarizing text and PDF documents using multimodal models.

In [0]:
%pip install pypdf2
%restart_python

In [0]:
import os
import time
import PyPDF2
import requests

In [0]:
from pyspark.sql import functions as F

doc_df = (
    spark.read
    .format("text")
    .option("wholetext", "true")
    .load(f"/Volumes/shm/bixi/raw_data/scrape/*.md")
    .withColumn("unique_id", F.monotonically_increasing_id())
    .select(
        "unique_id",
        F.col("_metadata.file_path").alias("file_name"),
        "value"
    )
)

display(doc_df)

In [0]:
%sql
SELECT
  filename,
  doc,
  count_tokens(doc) AS token_count,
  CURRENT_TIMESTAMP AS start_time,
  AI_QUERY(
    'serving-endpoint-name',
    CONCAT('Summarize this document in a single sentence', doc)
  ) AS llm_output,
  CURRENT_TIMESTAMP AS end_time,
  (unix_timestamp(end_time) - unix_timestamp(start_time)) AS inference_seconds
FROM shm.bixi.raw_data